In [1]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import seaborn as sns
import time

from nltk import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nltk.download("stopwords")

C:\Users\rahat\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
reviews = pd.read_csv('amazon_reviews_us_Baby_v1_00.tsv', sep='\t', on_bad_lines="warn")

b'Skipping line 9553: expected 15 fields, saw 22\nSkipping line 16240: expected 15 fields, saw 22\nSkipping line 21292: expected 15 fields, saw 22\nSkipping line 43160: expected 15 fields, saw 22\n'
b'Skipping line 75057: expected 15 fields, saw 22\nSkipping line 77996: expected 15 fields, saw 22\nSkipping line 90688: expected 15 fields, saw 22\n'
b'Skipping line 149039: expected 15 fields, saw 22\nSkipping line 156530: expected 15 fields, saw 22\nSkipping line 165609: expected 15 fields, saw 22\nSkipping line 174362: expected 15 fields, saw 22\nSkipping line 191190: expected 15 fields, saw 22\n'
b'Skipping line 201182: expected 15 fields, saw 22\nSkipping line 206503: expected 15 fields, saw 22\nSkipping line 231942: expected 15 fields, saw 22\nSkipping line 243480: expected 15 fields, saw 22\nSkipping line 252489: expected 15 fields, saw 22\n'
b'Skipping line 335656: expected 15 fields, saw 22\nSkipping line 355068: expected 15 fields, saw 22\nSkipping line 374843: expected 15 fields

In [3]:
reviews

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,9970739,R8EWA1OFT84NX,B00GSP5D94,329991347,Summer Infant SwaddleMe Adjustable Infant Wrap...,Baby,5,0.0,0.0,N,Y,Great swaddled blankets,Loved these swaddle blankets and so did my dau...,2015-08-31
1,US,23538442,R2JWY4YRQD4FOP,B00YYDDZGU,646108902,Pacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided...,Baby,5,0.0,0.0,N,N,Too cute and really nice,These are adorable pacifier clips. SavvyBaby h...,2015-08-31
2,US,8273344,RL5ESX231LZ0B,B00BUBNZC8,642922361,Udder Covers - Breast Feeding Nursing Cover,Baby,5,0.0,0.0,N,Y,Five Stars,Great gift,2015-08-31
3,US,24557753,RRMS9ZWJ2KD08,B00AWLZFTS,494272733,Gerber Graduates Fun Pack Utensils,Baby,5,0.0,0.0,N,Y,Cute; wash up nicely in dishwasher.,These forks are great for my 10 month old daug...,2015-08-31
4,US,46263340,R14I3ZG5E6S7YM,B00KM60D3Q,305813185,Summer Infant Ultra Sight Pan/Scan/Zoom Video ...,Baby,5,0.0,0.0,N,Y,Love it!,I wanted something for piece of mind with my l...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749143,US,52873209,R7W6TPBOZ7WJZ,B00000J0HR,865087255,Sassy Smiley Face Rattle,Baby,5,2.0,2.0,N,N,Great toy,My 6 month old son loves to play with this toy...,1999-07-14
1749144,US,51161415,R1Y4I7KYF7Y5MH,B00000J0HU,640300757,Sassy 836 - Fascination Station (Discontinued ...,Baby,5,0.0,0.0,N,N,a must have for active babies,Not only does my son love this toy but so do I...,1999-07-14
1749145,US,52165416,R35M9WTWVWMLX9,B00000ISE2,455759081,Neurosmith - Music Blocks with Mozart Music Ca...,Baby,5,27.0,27.0,N,N,"Great Music, Great Fun",This toy offers a fun learning experience and ...,1999-07-13
1749146,US,52784607,R2LUF1DDNNRNAJ,B00000DMCK,608283258,Lamaze Grip & Grab Rattle (Discontinued by Man...,Baby,5,18.0,18.0,N,N,The rattle that keeps up with your childs abil...,Our three-month-old had fun with this as soon ...,1999-07-13


In [4]:
reviews.iloc[0]['review_body']

'Loved these swaddle blankets and so did my daughter, she slept so much better when tightly swaddled. Only problem was we had a very hard time getting her to sleep without it when she started rolling.'

In [5]:
print(reviews.columns.tolist())

['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']


In [28]:
sample = reviews.sample(10000)

#Adjust sample size for accuracy

In [29]:
sample = sample.dropna()
sample['review_body'] = sample['review_body'].str.lower()
sample['review_body'] = sample['review_body'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [30]:
sample

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
1002005,US,33845162,R258VDVD0JZPVU,B000HEE73K,780355610,Kiddopotamus Cushystraps Super Soft Strap Cove...,Baby,4,0.0,0.0,N,Y,Very cute straps!,these are very cute strap covers but a little ...,2013-12-07
1504552,US,44600890,R2SPHL5C8QTBFU,B004IEBOVS,452950600,"GracoTravel Lite Crib With Stages, Notting Hil...",Baby,4,5.0,7.0,N,Y,Great Buy,after doing a lot of research on bassinets i d...,2011-04-21
1532217,US,30402156,R2NQ94Y7XGF0R7,B0011URFRE,800046243,Graco Nautilus 3-in-1 Car Seat,Baby,4,0.0,0.0,N,Y,carseat that works great for us!,we have liked our first graco seat that we bou...,2011-01-15
119011,US,21910008,R3VQ42YXCGAI0S,B0038JDAJ8,29833239,Vulli So` Pure Teether,Baby,4,0.0,0.0,N,Y,Four Stars,i thought is was bigger,2015-07-10
1700771,US,13647273,R1VKJ7KTU27L8I,B0009PAO20,915350324,Boudreaux's Butt Paste & Bobble Head Gift Set,Baby,5,1.0,1.0,N,N,The best around!!!!!!!!!!,ive tried them all i am a mother of 2 toddler...,2005-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36908,US,5396728,R1J0ZGVKUZ1PU9,B00UNPIP6S,135134703,"Re-Play Divided Plates, Purple/Green/Sunny Yel...",Baby,5,0.0,0.0,N,Y,Five Stars,it was pretty deep where it was enough to fulf...,2015-08-15
323090,US,35853684,R18ZAFDWWF5ZXL,B00HI4FKAQ,403508890,Motorola TFL3X44AAA900 Battery - Replacement f...,Baby,5,0.0,0.0,N,N,"Great Price, works well",this battery was a great price and works well ...,2015-03-28
1510750,US,28093458,R39G3QDKV1E1GT,B003DNS9XG,113198153,"Mommy's Helper Bath Mat Froggie Collection, Gr...",Baby,5,1.0,1.0,N,N,Perfect for kitchen sink!,this is the perfect solution to a slippery kit...,2011-04-01
197178,US,13868885,R1OSAQKGN0MWTO,B006ZVV0NO,513427451,"Britax EZ-Cling Sun Shades, Black, 2 Count",Baby,4,0.0,0.0,N,Y,Four Stars,its ok unsure about uv protection,2015-06-01


In [31]:
sample['star_rating'].values

array([4, 4, 4, ..., 5, '4', '5'], dtype=object)

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer

text_data = sample['review_body'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

In [33]:
sequences = tokenizer.texts_to_sequences(text_data)

In [34]:
sequences

[[39,
  22,
  28,
  146,
  477,
  491,
  16,
  6,
  56,
  158,
  5,
  144,
  728,
  43,
  4,
  10,
  535,
  561,
  309,
  1411,
  4,
  384,
  84,
  25,
  22,
  28,
  145,
  2,
  10,
  82,
  94,
  4,
  1195,
  15,
  43,
  12,
  1,
  92,
  6,
  30,
  5280,
  8,
  92,
  1536],
 [87,
  674,
  6,
  188,
  11,
  949,
  15,
  6080,
  5,
  426,
  15,
  1,
  551,
  3272,
  766,
  67,
  11,
  6,
  2694,
  59,
  6,
  671,
  3,
  441,
  59,
  338,
  2045,
  2,
  18,
  52,
  4,
  78,
  262,
  3,
  200,
  41,
  1,
  108,
  150,
  8,
  29,
  740,
  27,
  300,
  423,
  295,
  20,
  5,
  17,
  295,
  4,
  40,
  3,
  16,
  20,
  234,
  3,
  200,
  41,
  3,
  199,
  4,
  124,
  379,
  41,
  5,
  264,
  264,
  4,
  4699,
  1,
  1786,
  59,
  179,
  1572,
  13,
  1,
  2578,
  356,
  380,
  196,
  207,
  1,
  273,
  284,
  2,
  1,
  9750,
  478,
  233,
  1,
  624,
  273,
  6,
  56,
  65,
  204],
 [19,
  17,
  418,
  38,
  101,
  551,
  54,
  13,
  19,
  75,
  107,
  257,
  492,
  8,
  38,
  372,
  151,
  2,

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length)

In [36]:
from sklearn.model_selection import train_test_split

X = padded_sequences
y = sample['star_rating'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(LSTM(100))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')

In [38]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=256)

#Adjust batch size for accuracy

Epoch 1/10
 3/32 [=>............................] - ETA: 3:24 - loss: 1.7085

In [18]:
model.save('trained_reviews_model2.h5')

#Saves the model in hdf5 format for testing with frontend

C:\Users\rahat\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
